In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
from numpy import genfromtxt
import numpy as np
import os
import argparse
import sys


# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()


INPUTDIR='rnn_31_20s_onehot_nor_trainingdata_100subjects.csv'
MODELDIR="D:/dnn_SAS/model_new_test/first_model.ckpt"
GRAPHDIR="D:/dnn_SAS/model_new_test/"

#########################
## input data from csv ##
#########################
## 36562 ##
data = genfromtxt(INPUTDIR, delimiter=',')
np.random.shuffle(data)
# training data 
data_train=data[0:35500,:]
print(data_train.shape)
# np.random.shuffle(data_train)
data_x = data_train[0:35500,0:360]
data_y = data_train[0:35500,360:364]
train_x = data_train[0:15000,0:360]
train_y = data_train[0:15000,360:364]


# validation dataset 
val_data_x = data[35500:50700,0:360]
val_data_y = data[35500:50700,360:364]
print(val_data_x.shape)


sess = tf.InteractiveSession()

#####################
### visualization ###
#####################
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)

    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

#####################
## RNN Architectur ##
#####################
def RNN(inputs, weights, biases, name="RNN"):
    with tf.name_scope(name):
        
        with tf.name_scope('reshape_x'):
            X = tf.reshape(inputs, [-1, n_input])
            X_in = tf.matmul(X, weights['in']) + biases['in'] 
            X_in_ = tf.reshape(X_in, [-1, time_steps, num_units])
            
            variable_summaries(weights['in'])
            variable_summaries(biases['in'])
    
    # lstm cell
        with tf.variable_scope('lstm_cell'):
            lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_units, forget_bias=1.0)
            lstm_cell_drop = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
#         init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    
    # built LSTM-RNN 
        with tf.variable_scope('lstm_rnn'):   
            outputs1, state = tf.nn.dynamic_rnn(lstm_cell_drop, X_in_, dtype=tf.float32, time_major=False)
            result=tf.matmul(state[1],weights['out'])+biases['out']
            
            variable_summaries(weights['in'])
            variable_summaries(biases['in'])
        tf.summary.histogram('rnn_output',state[1])    
            
    return result


#######################
## define parameters ##
#######################
time_steps=40       # unrolled through 20 data step time(10s)
num_units=80       # hidden LSTM units
n_input=9           # features
learning_rate=0.001 # learning rate for adam
n_classes=4         # classified in 4 classes 
batch_size=500
beta=0.005              # 0.005
epoch=3000                 # epoch
best_acc_valid = 0         # best accuracy of validation dataset
acc_pause_num=0            
n_improve_threshold = 200  # threshold for early stop



########################################
## weights for input and output layer ##
########################################

weights = {
    'in': tf.Variable(tf.random_normal([n_input,num_units])),    # (6, 36)    
    'out': tf.Variable(tf.random_normal([num_units, n_classes])) # (36, 4)
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[num_units, ])),    # (36, )
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))    # (4, )
}


#######################
## Input Placeholder ##
#######################

x=tf.placeholder("float",[None,time_steps,n_input],name="x_input")
y=tf.placeholder("float",[None,n_classes],name="y_input")
keep_prob = tf.placeholder(tf.float32)

pred = RNN(x, weights, biases)

# output = tf.argmax(pred,1,name="output")

with tf.name_scope("loss"):
    with tf.name_scope("loss1"):
#     loss1=tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred),reduction_indices=[1]),name="loss1")
        loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y), name="loss1")
    tf.summary.scalar("loss_cross", loss1)
    
    with tf.name_scope("loss1"):
        #regularizer
        tv = tf.trainable_variables()
        l2 = beta * sum(
            tf.nn.l2_loss(tv)
                for tv in tf.trainable_variables()
                if not ("noreg" in tv.name or "Bias" in tv.name)
        )
        loss2=loss1+l2
    tf.summary.scalar("loss_l2reg", loss2)
    
with tf.name_scope("train"):
    opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss2)
    
    
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="accuracy")
tf.summary.scalar("accuracy", accuracy)
    
    
summ = tf.summary.merge_all()


# Save model
saver = tf.train.Saver()

# initialization
init = tf.global_variables_initializer()


with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(GRAPHDIR + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(GRAPHDIR + '/test')
    sess.run(init)
    train_writer.add_graph(sess.graph)
    test_writer.add_graph(sess.graph)
    
    for e in range(epoch):
        i = 0
        while i < len(data_y):
            # Using different batch for every iteration
            start = i
            end = i + batch_size
        
            batch_x = data_x[start:end]
            batch_y = data_y[start:end]
            
            batch_x=np.reshape(batch_x,(batch_size,time_steps,n_input))

            sess.run(opt, feed_dict={x: batch_x, y: batch_y,  keep_prob: 1})
            i += batch_size

        #######################################################
        ## print training data, valid data accuracy and loss ##
        #######################################################
        
        if e % 10==0:
            train_x_=np.reshape(train_x,(15000,time_steps,n_input))
            train_acc, s1, train_loss = sess.run([accuracy, summ, loss2],feed_dict={x: train_x_, y: train_y, keep_prob: 1})
            train_writer.add_summary(s1, e)
            print("Iter ",e)
            print("Accuracy: ",train_acc)
            print("Loss: ",train_loss)
#             print("__________________")
            
        if e % 10==0:
            val_data_x=np.reshape(val_data_x,(15200,time_steps,n_input))
            [valid_acc, s2, valid_loss]=sess.run([accuracy, summ, loss2], feed_dict={x:val_data_x,y:val_data_y,keep_prob: 1})
#             los=sess.run(loss,feed_dict={x:np_val_data_x,y:np_val_data_y,keep_prob: 1})
#             print("For iter ",e)
            test_writer.add_summary(s2, e)
            print("Valid Accuracy: ",valid_acc)
            print("Valid Loss: ",valid_loss)
            print("__________________")
            
        
        
        #############################################
        ## save the best model that fit valid data ##
        #############################################
#         if valid_acc > best_acc_valid:
#             best_acc_valid = valid_acc
#             save_path = saver.save(sess, "D:/dnn_SAS/model_new_test/first_3_1_model.ckpt")
#             acc_pause_num = 0
#         else:
#             acc_pause_num += 1
            
#         # Stop the training operation if accuracy odoes not improve over 100 epochs
#         if acc_pause_num >= n_improve_threshold:

#             break    
            
            
train_writer.close()
test_writer.close()

(35500, 364)
(15200, 360)
Iter  0
Accuracy:  0.687266
Loss:  3.78982
Valid Accuracy:  0.692697
Valid Loss:  3.77482
__________________


KeyboardInterrupt: 

In [9]:
NUMS='nums.csv'
nums = genfromtxt(NUMS, delimiter=',')
file1 = 'final_test_data_003'
file2 = '.csv'
outfile = 'lstm_rnn_20s_public_result/out_20s'
outsoft = 'lstm_rnn_20s_public_result/out_soft_20s'
infile = 'lstm_rnn_20s_public_result/in_20s'
MODELDIR = "D:/dnn_SAS/model_new_test/first_3_1_model.ckpt"

P_NUM='003'

def softmaxx(z):
    assert len(z.shape) == 2
    s = np.max(z, axis=1)
    s = s[:, np.newaxis] # necessary step to do broadcasting
    e_x = np.exp(z - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis] # dito
    return e_x / div

# for i in range(len(nums)):
for i in range(1):
#     num=nums[i]
#     P_=num_str=np.array2string(num, precision=1)
#     P_NUM=P_[0:4]
#     patient = genfromtxt(file1 + P_NUM + file2, delimiter=',')
    patient = genfromtxt(file1 + file2, delimiter=',')
    [h,k]=(patient.shape)
    pat_x=patient[0:h,0:360]
    pat_y=patient[0:h,360:364]
    [a,b]=(pat_x.shape)
    print(a)
    print(b)
    # print(type(pat_y))
    ##########################################
    with tf.Session() as sess:
    
        load_path = saver.restore(sess, MODELDIR)
        pat_x_=np.reshape(pat_x,(a,time_steps,n_input))
        print(type(pat_x_))
        new_test_acc=sess.run(accuracy,feed_dict={x:pat_x_,y:pat_y})
    
        prediction =sess.run(pred, feed_dict={x:pat_x_})
        soft_out=softmaxx(prediction)
        out=np.argmax(prediction,1)
        np.savetxt(outfile + P_NUM + file2, out, delimiter = ',')
        np.savetxt(outsoft + P_NUM + file2, soft_out, delimiter = ',')
        input_matrix=np.argmax(pat_y,1)
        np.savetxt(infile + P_NUM + file2, input_matrix, delimiter = ',')
    
#     los=sess.run(loss,feed_dict={x:test_data_x,y:test_data_y})
        print("Test Accuracy: ",new_test_acc)
        print("__________________")

1282
360
<class 'numpy.ndarray'>
Test Accuracy:  0.659906
__________________


In [14]:
a=[1,2,3]
softmaxx(a)

AttributeError: 'list' object has no attribute 'shape'